In [1]:
import glob
import pandas as pd
import numpy as np
import os
from IPython.display import HTML

In [2]:
!ls /home/pierre/scVI/results/models_comparison/

mixed_25_dataset  mixed_50_dataset  mixed_75_dataset  nb_dataset  zinb_dataset


In [3]:
data_files = glob.glob('/home/pierre/scVI/results/models_comparison/{}/*.csv'.format('mixed_50_dataset'))
data_folder = '/home/pierre/scVI/results/models_comparison/{}'.format('mixed_50_dataset')

data_names = ['zifa_full', 'nb', 'zinb']

In [4]:
dfs = []
for data_name, f in zip(data_names, data_files):
    my_df = pd.read_csv(f, sep='\t')
    my_df.loc[:, 'model'] = data_name
#     my_df.columns = [data_name + '_' col_name for col_name in my_df.columns]
    dfs.append(my_df)
df = pd.concat(dfs, axis=0)
# df.info()

ValueError: No objects to concatenate

In [ ]:
metrics = ['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat', 't_ratio_ks_stat', 't_cv_ks_stat']
h1_hypothesis = ['larger', 'larger', 'larger', 'larger', 'larger']
df_nb = df.loc[df.model=='nb', metrics]

from statsmodels.stats.weightstats import ttest_ind

def get_pvals(gby):
    my_df = gby[metrics]
    assert my_df.shape[1] == len(metrics)
    pvals = []
    for idx, alternative in enumerate(h1_hypothesis):
        assert len(df_nb.values[:, idx]) != len(h1_hypothesis)
        _, pval, _ = ttest_ind(df_nb.values[:, idx], my_df.values[:, idx], alternative=alternative)
        pvals.append(pval)
    return np.array(pvals)

pvals = df.groupby('model').apply(get_pvals)
pvals = (pvals
         .apply(lambda x: pd.Series(x))
         .T)
pvals = pvals.rename(index={idx: met for (idx,met) in enumerate(metrics)})

In [ ]:
def get_summary(gby):
    res = {}
    res['mean'] = gby.mean()
    res['std'] = gby.std()
    res['pvals'] = pvals[gby.name]
    return pd.DataFrame(res).T

# df_summary = df.groupby('model')['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat',
#                                  't_ratio_ks_stat', 't_cv_ks_stat'].agg(['mean', 'std']).T
df_summary = df.groupby('model')[metrics].apply(get_summary)
df_summary


def my_styler(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    print(val)
    style = 'background-color: yellow' if val.pvals < 0.05 else ''
    return [style]

df_summary = df_summary.stack().unstack(1).sort_index(level=1).swaplevel()
df_summary.style.apply(my_styler, subset=['pvals'], axis=1)
# df_summary.loc[pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :]].style.background_gradient()
# df_summary.style.apply(my_styler, subset=pd.IndexSlice[:, 'pvals'], axis=0)
# df_summary = df_summary.style.apply(my_styler, subset=pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :], axis=1).render()
# df_summary.pivot(index=['model', ['mean', 'std', 'pvals']], columns=metrics)

In [ ]:
# df_summary.loc[pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :]].style.background_gradient()

In [ ]:
text_file = open(os.path.join(data_folder, 'latex_table.txt'), "w")
text_file.write(df_summary.to_latex())
text_file.close()